In [16]:
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [17]:
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10), place idx에서 23개가 비어있음
product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5834,11), product idx에서 538개가 비어있음
video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9), video idx에서 315개가 비어있음
index_null_del_user_place = pd.read_csv('../Data/whyout_data/index_null_del_user_place.csv', index_col=0).astype(dtype='float16')
index_null_del_user_product = pd.read_csv('../Data/whyout_data/index_null_del_user_product.csv', index_col=0).astype(dtype='float16')
index_null_del_user_video = pd.read_csv('../Data/whyout_data/index_null_del_user_video.csv', index_col=0).astype(dtype='float16')

In [ ]:
class MF():
    def __init__(self,ratings,hyper_params):
        self.R = np.array(ratings)
        #사용자 수(num_users)와 아이템 수 (num_items)를 받아온다.
        self.num_users, self.num_items = np.shape(self.R)
        #아래는 MF weight 조절을 위한 하이퍼 파라미터
        # K : 잠재요인(Latent Factor)의 수
        self.K = hyper_params['K']
        # alpha : 학습률
        self.alpha = hyper_params['alpha']
        # beta : 정규화 계수
        self.beta = hyper_params['beta']
        # iterations : SGD 계산을 할 때의 반복 횟수
        self.iterations = hyper_params['iterations']
        # verbose : SGD의 학습 과정을 중간중간에 출력할 것인지에 대한 여부
        self.verbose = hyper_params['verbose']

    
    #P와 Q를 가지고 RMSE 함수 형성
    def rmse(self):
        #self.R의 rating 데이터 속에서 0이 아닌 요소에 인덱스 가져오기
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []

        for x, y  in zip(xs, ys):
            #get_prediction은 사용자 x, 아이템 y에 대해서 평점 예측치를 계산하는 class method
            prediction = self.get_prediction(x,y)
            self.predictions.append(prediction)

            #실측값과 예측값의 차이를 리스트에 어펜드
            self.errors.append(self.R[x,y] - prediction)
        #방금 한 것을 array 형태로 다시 바꿔준다
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)

        #우리가 알고 있는 rmse를 클래스 매서드로 구현
        return np.sqrt(np.mean(self.errors**2))
        

    #학습에 대한 method 구현
    def train(self):
        #P행렬과 Q행렬 난수 초기화
        #표준편차 = 잠재변수의 개수 분의 1
        self.P = np.random.normal(scale=1./self.K,
                                  size = (self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K,
                                  size = (self.num_items, self.K))
       
        #사용자 평가 경향도 초기화; 0으로부터 시작
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        rows, columns = self.R.nonzero()
        
        #SGD를 적용할 대상; 평점이 있는 요소의 인덱스와 그 평점을 리스트로 만들어서 samples에 저장
        self.samples = [(i,j,self.R[i,j]) for i, j in zip(rows, columns)]

        training_process = []
        for i in range(self.iterations):

            #다양한 시작점에서 계속 shuffle를 하면서 진행
            np.random.shuffle(self.samples)
            self.sgd()
          
            #SGD로 P와 Q와 B_u, B_d가 업데이트되었으므로 이에 따른 새로운 RMSE 계산
            rmse = self.rmse()
            training_process.append((i+1, rmse))
            if self.verbose:
              if (i+1) % 10 == 0:
                print('Iteration : %d ; train RMSE = %.4f' % (i+1, rmse))
        
        return training_process 

    #i 사용자의 j 아이템에 대한 평점 예측
    def get_prediction(self, i, j):
        #r_hat = 전체 평점 + 사용자 유저에 대한 평가 경향 + 아이템에 대한 평가 경향 + 사용자 요인에 대해서 i번째 사용자에 대한 요인값
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i,:].dot(self.Q[j,].T)
        return prediction

    #sgd 구하기 : 최적의 P, Q, b_u, b_d 구하기
    def sgd(self):
        for i, j, r in self.samples:
          prediction = self.get_prediction(i,j)
          #오차 계산) 실제값과 예측값의 차이
          e = r - prediction
          #사진 1 참고
          self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])  
          self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])
          #사진 2 참고
          self.P[i,:] += self.alpha * ((e * self.Q[j,:]) - (self.beta * self.P[i,:]))
          self.Q[j,:] += self.alpha * ((e * self.P[i,:]) - (self.beta * self.Q[j,:]))

R_temp = index_null_del_user_place

hyper_params = {
    'K' : 30,
    'learning_rate' : 0.001,
    'reg_param' : 0.02,
    'iterations' : 100,
    'verbose' : True
}

mf = MF(R_temp, hyper_params)

train_process = mf.train()

In [19]:
from sklearn.utils import shuffle #데이터가 imbalance하면 계층화추출, balance하면 셔플(계층화 추출하면, 표본의 대표성을 저해할 수 있기 때문)
TRAIN_SIZE = 0.75

ratings = shuffle(index_null_del_user_place,random_state=2021)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

In [20]:
class NEW_MF():
  def __init__(self,ratings,hyper_params):
    self.R = np.array(ratings)
    #사용자 수(num_users)와 아이템 수 (num_items)를 받아온다.
    self.num_users,self.num_items = np.shape(self.R)
    #아래는 MF weight 조절을 위한 하이퍼 파라미터
    # K : 잠재요인(Latent Factor)의 수
    self.K = hyper_params['K']
    self.alpha = hyper_params['alpha']
    self.beta = hyper_params['beta']
    self.iterations = hyper_params['iterations']
    self.verbose = hyper_params['verbose']

    # 데이터가 연속적이지 않을 때, 인덱스와 맞지 않는 것을 방지하기위해 맵핑해주는 작업
    item_id_index = []
    index_item_id = []

    for i, one_id in enumerate(ratings):
      item_id_index.append([one_id, i])
      index_item_id.append([i, one_id])
    self.item_id_index = dict(item_id_index)
    self.index_item_id = dict(index_item_id)

    user_id_index = []
    index_user_id = []

    for i, one_id in enumerate(ratings.T):
      user_id_index.append([one_id, i])
      index_user_id.append([i, one_id])
    self.user_id_index = dict(user_id_index)
    self.index_user_id = dict(index_user_id)


  def rmse(self):
    # self.R에서 평점이 있는(0이 아닌) 요소의 인덱스를 가져온다.
    xs,ys = self.R.nonzero()
    # prediction과 error를 담을 리스트 변수 초기화
    self.predictions = []
    self.errors = []
    # 평점이 있는 요소(사용자 x, 아이템 y) 각각에 대해서 아래의 코드를 실행
    for x,y in zip(xs,ys):
      #사용자 x, 아이템 y에 대해서 평점 예측치를 get_prediction()함수를 사용해서 계산
      prediction=self.get_prediction(x,y)
      # 예측값을 예측값 리스트에 추가
      self.predictions.append(prediction)
      # 실제값(R)과 예측값의 차이(errors) 계산해서 오차값 리스트에 추가
      self.errors.append(self.R[x,y] - prediction)
    #예측값 리스트와 오차값 리스트를 numpy array형태로 변환
    self.predictions = np.array(self.predictions)
    self.errors = np.array(self.errors)
    #error를 활용해서 RMSE 도출
    return np.sqrt(np.mean(self.errors**2))

  def sgd(self):
    for i,j,r in self.samples:
      #사용자 i 아이템 j에 대한 평점 예측치 계산
      prediction = self.get_prediction(i,j)
      # 실제 평점과 비교한 오차 계산
      e = (r - prediction)

      # 사용자 평가 경향 계산 및 업데이트
      self.b_u[i] += self.alpha *(e - (self.beta * self.b_u[i]))
      # 아이템 평가 경향 계산 및 업데이트
      self.b_d[j] += self.alpha *(e - (self.beta * self.b_d[j]))
      # P 행렬 계산 및 업데이트
      self.P[i,:] += self.alpha * ((e * self.Q[j,:]) - (self.beta * self.P[i,:]))
      # Q 행렬 계산 및 업데이트
      self.Q[j,:] += self.alpha * ((e * self.P[i,:]) - (self.beta * self.Q[j,:]))

  def get_prediction(self,i,j): #평점 예측값 구하는 함수
    #전체 평점 + 사용자 평가 경향 + 아이템에 대한 평가 경향 + i번째 사용자의 요인과 j번째 아이템 요인의 행렬 곱
    prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i,:].dot(self.Q[j,:].T) 
    return prediction
  
  # Test Set 선정
  def set_test(self,ratings_test):
    if ratings_test.shape[1] != 3:
      print("ratings_test가 예상한 형식이 아닙니다. 변환을 시도합니다.")
      # 필요하면 적절한 형식으로 변환하는 코드를 작성
      # 예를 들어, 첫 번째 열이 user_id, 두 번째 열이 item_id, 세 번째 열이 rating이 되도록 변환
      ratings_test = ratings_test.reset_index().melt(id_vars=['index'], var_name='item_id', value_name='rating')
      ratings_test.columns = ['user_id', 'item_id', 'rating']
      ratings_test = ratings_test[ratings_test['rating'] != 0]  # 평점이 0이 아닌 데이터만 선택
    test_set = []
    for i in range(len(ratings_test)):
      x = self.user_id_index[ratings_test.iloc[i,0]] # 사용자 id
      y = self.item_id_index[ratings_test.iloc[i,1]] # 아이템 id
      z = ratings_test.iloc[i,2] # 평점
      test_set.append([x,y,z])
      self.R[x,y] = 0
    self.test_set = test_set
    return test_set

  # Test Set RMSE 계산
  def test_rmse(self):
    error = 0
    for one_set in self.test_set:
      predicted = self.get_prediction(one_set[0], one_set[1])
      error += pow(one_set[2] - predicted, 2)
    return np.sqrt(error/len(self.test_set))

  def test(self):
    self.P = np.random.normal(scale=1./self.K,
                                  size = (self.num_users, self.K))
    self.Q = np.random.normal(scale=1./self.K,
                              size = (self.num_items, self.K))
    
    #사용자 평가 경향도 초기화; 0으로부터 시작
    self.b_u = np.zeros(self.num_users)
    self.b_d = np.zeros(self.num_items)
    self.b = np.mean(self.R[self.R.nonzero()])

    rows, columns = self.R.nonzero()
    self.samples = [(i,j, self.R[i,j]) for i,j in zip(rows, columns)]

    training_process = []
    for i in range(self.iterations):
      np.random.shuffle(self.samples)
      self.sgd()
      rmse1 = self.rmse()
      rmse2 = self.test_rmse()
      training_process.append((i+1, rmse1, rmse2))
      if self.verbose:
        if (i+1) % 10 == 0:
          print('Iteration : %d ; Train RMSE = %.4f ; Test RMSE = %4f'% (i+1, rmse1, rmse2))
    return training_process

  def get_one_prediction(self, user_id, item_id):
    return self.get_prediction(self.user_id_index[user_id],
                               self.item_id_index[item_id])
    
  def full_prediction(self):
    return self.b + self.b_u[:, np.newaxis] + self.b_d[np.newaxis, :] + self.P.dot(self.Q.T)

R_temp = index_null_del_user_place

hyper_params = {
    'K' : 30,
    'alpha' : 0.001,
    'beta' : 0.02,
    'iterations' : 100,
    'verbose' : True
}

mf = NEW_MF(R_temp, hyper_params)
test_set = mf.set_test(ratings_test)
result = mf.test()

ratings_test가 예상한 형식이 아닙니다. 변환을 시도합니다.
Iteration : 10 ; Train RMSE = 0.8111 ; Test RMSE = 0.886567
Iteration : 20 ; Train RMSE = 0.7880 ; Test RMSE = 0.886396
Iteration : 30 ; Train RMSE = 0.7756 ; Test RMSE = 0.886547
Iteration : 40 ; Train RMSE = 0.7675 ; Test RMSE = 0.886668
Iteration : 50 ; Train RMSE = 0.7615 ; Test RMSE = 0.886842
Iteration : 60 ; Train RMSE = 0.7565 ; Test RMSE = 0.886963
Iteration : 70 ; Train RMSE = 0.7518 ; Test RMSE = 0.887088
Iteration : 80 ; Train RMSE = 0.7469 ; Test RMSE = 0.887169
Iteration : 90 ; Train RMSE = 0.7412 ; Test RMSE = 0.887301
Iteration : 100 ; Train RMSE = 0.7341 ; Test RMSE = 0.887456
